In [ ]:
!pip install scikit-multilearn
!pip install neattext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
'''
Мы использовали модель мультиномиального наивного Байеса (Multinomial Naive Bayes) вместе с методом преобразования проблемы Classifier Chain для решения задачи мультиклассовой классификации текста. В качестве предварительной обработки текста мы использовали различные методы из библиотеки Neattext, такие как удаление шума и стоп-слов. Для векторизации текстовых данных в числовой формат мы использовали метод TF-IDF.

Перед обучением модели мы преобразовали метки классов в удобный формат и создали столбцы для каждой метки. Далее разделили данные на обучающий и тестовый наборы.

Модель мультиномиального наивного Байеса была обучена на обучающем наборе данных с использованием TF-IDF преобразования для векторизации текста. Затем мы делали предсказания на тестовом наборе данных и оценивали производительность модели с помощью F1-меры.
'''
import pandas as pd
import ast
import random
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.problem_transform import ClassifierChain
import neattext as nt
import neattext.functions as nfx
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline

# Определяем размер чанка для обработки данных порциями
chunksize = 5000
# Загружаем обучающий набор данных из CSV файла
df = pd.read_csv('drive/MyDrive/train2.csv')

# Функция для извлечения меток из строки, представленной в виде списка
def extract_labels(label_str):
    label_list = ast.literal_eval(str(label_str))
    return list(label_list)

# Применяем функцию извлечения меток к столбцу 'labels'
df['labels'] = df['labels'].apply(extract_labels)

# Удаляем шум и стоп-слова из текста
df['text'].apply(lambda x: nt.TextFrame(x).noise_scan())

df['text'].apply(lambda x: nt.TextExtractor(x).extract_stopwords())

df['text'].apply(nfx.remove_stopwords)

# Создаем список всех возможных меток
all_labels = ['work', 'news', 'sports', 'music', 'movies',
              'politics', 'phones', 'self-driving cars',
              'family', 'cars', 'climate change', 'languages',
              'business', 'health', 'science', 'style', 'opinion',
              'economy', 'history', 'technology', 'affair', 'development', 'mobility']

for label in all_labels:
    df[label] = 0.0

for i, row in df.iterrows():
    for label in row['labels']:
        df.at[i, label] = 1.0

# Определяем целевые метки
y = df[['work', 'news', 'sports', 'music', 'movies',
        'politics', 'phones', 'self-driving cars',
        'family', 'cars', 'climate change', 'languages',
        'business', 'health', 'science', 'style', 'opinion',
        'economy', 'history', 'technology', 'affair', 'development', 'mobility']]

# Разделяем данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(df["text"], y, test_size=0.3, random_state=42)

# Создаем конвейер для обработки данных и обучения модели
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', ClassifierChain(MultinomialNB())),
])

# Обучаем конвейер на обучающих данных
pipeline.fit(X_train, y_train)
# Делаем предсказания на тестовых данных
clf_predictions = pipeline.predict(X_test)
# Выводим F1-меру
print(f1_score(y_test, clf_predictions, average='weighted', zero_division=0))

# Функция для преобразования предсказаний в объекты
def convert_predictions_to_objects(predictions, labels):
    predicted_objects_set = set()

    for instance_predictions in predictions:
        instance_objects = []

        for prediction, label in zip(instance_predictions, labels):
            if prediction == 1.0:
                instance_objects.append(label)

        predicted_objects_set.update(instance_objects)

    if len(predicted_objects_set) == 0:
        predicted_objects_set.update({random.choice(all_labels)})

    return predicted_objects_set

# Обработка данных тестового набора и создание предсказаний
for chunk in pd.read_csv('drive/MyDrive/test2.csv', chunksize=chunksize):
    chunk.text = chunk.text.astype(str)
    chunk['text'].apply(nfx.remove_stopwords)
    chunk['text'].apply(lambda x: nt.TextFrame(x).noise_scan())
    chunk['text'].apply(lambda x: nt.TextExtractor(x).extract_stopwords())
    clf_predictions = pipeline.predict(chunk["text"])
    new_pred = []

    for pred in clf_predictions:
        new_pred.append(convert_predictions_to_objects(pred.toarray(), all_labels))

    results_df = pd.DataFrame({
        '': chunk.index,
        'labels': new_pred
    })

    results_df.to_csv("results.csv", index=False, mode='a',
                      header=False)

print(results_df.head())